# Normalizing Flows

![Status](https://img.shields.io/static/v1.svg?label=Status&message=WIP&color=orange)

**Author:** Julius Cathalina

**Last Updated (dd-mm-yyyy):** 29-01-2025

In [ ]:
## Standard libraries
import os
import math
import time
import numpy as np 

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

## Progress bar
from tqdm.notebook import tqdm


: 